# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/scw-comp/

In [1]:
import unittest
import re

## eventTitles

Proper names for the events - short, medium and long format

In [2]:
eventNames = \
[
    '333',
    '222',
    '444',
    '555',
    '666',
    '777',
    '333oh',
    'minx',
    'pyram',
    'skewb',
    'sq1',
    'clock',
    '333bf',
    '444bf',
    '555bf',
    '333fm'
]

eventTitles = \
{
    '333': ['3x3x3', '3x3x3', '3x3x3'],
    '222': ['2x2x2', '2x2x2', '2x2x2'],
    '333oh': ['3x3x3 One-Handed', '3x3x3 OH', 'OH'],
    'minx':  ['Megaminx', 'Megaminx', 'Mega'],

    '444': ['4x4x4', '4x4x4', '4x4x4'],
    '555': ['5x5x5', '5x5x5', '5x5x5'],
    '666': ['6x6x6', '6x6x6', '6x6x6'],
    '777': ['7x7x7', '7x7x7', '7x7x7'],

    'pyram': ['Pyraminx', 'Pyraminx', 'Pyra'],
    'skewb': ['Skewb', 'Skewb', 'Skewb'],
    'sq1': ['Square-1', 'Square-1', 'Sq-1'],
    'clock': ['Clock', 'Clock', 'Clock'],

    '333bf': ['3x3x3 Blindfolded', '3x3x3 BLD', '3BLD'],
    '444bf': ['4x4x4 Blindfolded', '4x4x4 BLD', '4BLD'],
    '555bf': ['5x5x5 Blindfolded', '5x5x5 BLD', '5BLD'],
    '333fm': ['3x3x3 Fewest Moves', '3x3x3 FMC', 'FMC']
}

## numSeconds()

Convert float or string to number of seconds (truncated to nearest 100th) - e.g. 1:05.319 returns 65.31

In [3]:
import math

def numSeconds(value, truncate = True):
    '''Convert float or string to number of seconds (truncated to nearest 100th) - e.g. 1:05.319 returns 65.31'''
    
    # String result (e.g. MM:SS.cc, SS.cc, nn, DNF or DNS)
    if isinstance(value, str):
        try:
            # Some people may have used commas instead of dots
            value = value.replace(',', '.')

            # Check that the result is either a time, DNF or DNS
            resultPattern = re.compile('^([1-7][0-9]|80)$|^([0-5]?[0-9]:[0-5]|[0-5])?[0-9]\.[0-9][0-9]$|^DNF$|^DNS$')
            if not resultPattern.match(value):
                raise ValueError(value, type(value))

            # MM:SS.cc
            if ':' in value:
                parts = value.split(':')
                value = int(parts[0]) * 60 + float(parts[1])
            # SS.cc
            elif '.' in value:
                value = float(value)
            # DNF
            elif value == 'DNF':
                return -1
            # DNS
            elif value == 'DNS':
                return -2
            # Assume nn - i.e. FMC
            else:
                return int(value)
        except:
            raise
    
    # Convoluted approach is required to handle imprecision of floating point arithmetic
    if truncate:
        return math.trunc(round(value * 1000) / 10) / 100
    else:
        return round(value, 2)

In [4]:
class TestNumSeconds(unittest.TestCase):
    '''Class to test numSeconds function'''   

    def test_simple_nn(self):
        '''Test simple numbers - e.g. FMC single'''
        self.assertEqual(numSeconds('20'), 20)

    def test_simple_sscc(self):
        '''Test simple times - ss.cc'''
        self.assertEqual(numSeconds('12.34'), 12.34)

    def test_simple_mmsscc(self):
        '''Test simple times - mm:ss.cc'''
        self.assertEqual(numSeconds('1:02.34'), 62.34)

    def test_problematic_mmsscc(self):
        '''Test problematic times (imprecision of floating point arithmetic) - mm:ss.cc'''
        self.assertEqual(numSeconds('5:25.09'), 325.09)
        self.assertEqual(numSeconds('1:40.23'), 100.23)
        self.assertEqual(numSeconds('2:08.70'), 128.70)

    def test_simple_int(self):
        '''Test simple numbers - e.g. FMC single'''
        self.assertEqual(numSeconds(20), 20)

    def test_simple_float(self):
        '''Test simple times - mm:ss.cc'''
        self.assertEqual(numSeconds(62.344), 62.34)
        self.assertEqual(numSeconds(62.345), 62.34)
        self.assertEqual(numSeconds(62.346), 62.34)

    def test_problematic_float(self):
        '''Test problematic times (imprecision of floating point arithmetic) - mm:ss.cc'''
        self.assertEqual(numSeconds(325.09), 325.09)
        self.assertEqual(numSeconds(100.23), 100.23)
        self.assertEqual(numSeconds(128.70), 128.70)

    def test_dnf(self):
        '''Test DNF code'''
        self.assertEqual(numSeconds('DNF'), -1)

    def test_dns(self):
        '''Test DNS code'''
        self.assertEqual(numSeconds('DNS'), -2)

    def test_xxx(self):
        '''Test unsupported code'''
        with self.assertRaises(ValueError):
            numSeconds('XXX')

## formatResult()

Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31

In [5]:
def formatResult(value, eventName = None, highlight = ''):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value is not None:
        if value > 0:
            if eventName == '333fm':
                return '{}{:d}{}'.format(highlight, int(value), highlight)
            else:
                if value >= 60:
                    return '{}{:d}:{:05.2f}{}'.format(highlight, int(value // 60), value - int(value // 60) * 60, highlight)
                else:
                    return '{}{:.2f}{}'.format(highlight, value, highlight)
        else:
            if value == -1:
                return 'DNF'
            elif value == -2:
                return 'DNS'
            else:
                raise ValueError(value, type(value))
    else:
        raise ValueError(value)

In [6]:
class TestFormatResult(unittest.TestCase):
    '''Class to test formatResult function'''   

    def test_fm(self):
        '''Test simple numbers - e.g. FMC single'''
        self.assertEqual(formatResult(20, eventName = '333fm'), '20')

    def test_time(self):
        '''Test simple times without highlighting - mm:ss.cc'''
        self.assertEqual(formatResult(1.23), '1.23')
        self.assertEqual(formatResult(12.34), '12.34')
        self.assertEqual(formatResult(62.34), '1:02.34')

    def test_highlight(self):
        '''Test simple times with highlighting - mm:ss.cc'''
        self.assertEqual(formatResult(1.23, highlight = '*'), '*1.23*')
        self.assertEqual(formatResult(12.34, highlight = '*'), '*12.34*')
        self.assertEqual(formatResult(62.34, highlight = '*'), '*1:02.34*')

    def test_dnf(self):
        '''Test DNF code'''
        self.assertEqual(formatResult(-1), 'DNF')

    def test_dns(self):
        '''Test DNS code'''
        self.assertEqual(formatResult(-2), 'DNS')

    def test_zero(self):
        '''Test Zero'''
        with self.assertRaises(ValueError):
            formatResult(0)

    def test_none(self):
        '''Test None'''
        with self.assertRaises(ValueError):
            formatResult(None)

## interpretAge()

Interpret age of <30, under 30, 40+, over 40, etc.

In [7]:
def interpretAge(age):
    '''Interpret age, however it is written and return an integer'''
    
    age = age.lower()
    if 'under' in age:
        age = int(age.replace('under', '').replace(' ', '')) - 10
    elif 'over' in age:
        age = int(age.replace('over', '').replace(' ', ''))
    elif '<' in age:
        age = int(age.replace('<', '').replace(' ', '')) - 10
    elif '>' in age:
        age = int(age.replace('>', '').replace(' ', ''))
    else:
        age = int(age.replace('+', ''))
        
    return int(age)

In [8]:
class TestInterpretAge(unittest.TestCase):
    '''Class to test interpretAge function'''   

    def test_under_40(self):
        '''Test under 40'''
        self.assertEqual(interpretAge('under 40'), 30)
        self.assertEqual(interpretAge('<40'), 30)

    def test_over_40(self):
        '''Test over 40'''
        self.assertEqual(interpretAge('over 40'), 40)
        self.assertEqual(interpretAge('>40'), 40)
        self.assertEqual(interpretAge('40+'), 40)

## formatAge() and formatAgeLong()

Format age to <30, 40+, etc.

In [9]:
def formatAge(age):
    '''Format age for report'''
    
    if age < 40:
        return '<{}'.format(age + 10)
    else:
        return '{}+'.format(age)

    
def formatAgeLong(age):
    '''Format age for report'''
    
    if age < 40:
        return 'Under {}'.format(age + 10)
    else:
        return 'Over {}'.format(age)

In [10]:
class TestFormatAge(unittest.TestCase):
    '''Class to test formatAge function'''   

    def test_under_40(self):
        '''Test under 40'''
        self.assertEqual(formatAge(20), '<30')
        self.assertEqual(formatAge(30), '<40')

    def test_over_40(self):
        '''Test over 40'''
        self.assertEqual(formatAge(40), '40+')
        self.assertEqual(formatAge(50), '50+')


class TestFormatAgeLong(unittest.TestCase):
    '''Class to test formatAgeLong function'''   

    def test_under_40(self):
        '''Test under 40'''
        self.assertEqual(formatAgeLong(20), 'Under 30')
        self.assertEqual(formatAgeLong(30), 'Under 40')

    def test_over_40(self):
        '''Test over 40'''
        self.assertEqual(formatAgeLong(40), 'Over 40')
        self.assertEqual(formatAgeLong(50), 'Over 50')

## formatFacebookLink()

Tidy up Facebook link - ensure it is not mobile specific and remove junk from the end

In [11]:
def formatFacebookLink(link):
    '''Change mobile links to regular Facebook link'''
    
    if link and link.startswith('http'):
        # Check that link is a permalink or video link
        resultPattern = re.compile('https:\/\/(www|m)\.facebook\.com\/((groups|events)\/[0-9]+.*permalink.*|[A-Za-z0-9\.]*\/videos.*)')
        if not resultPattern.match(link):
            raise ValueError(link)

        # Convert mobile links
        if '//m.' in link:
            link = link.replace('//m.', '//www.')
        if '?view=permalink&id=' in link:
            link = link.replace('?view=permalink&id=', '/permalink/')

        # Convert post
        if '?post_id=' in link:
            link = link.replace('?post_id=', 'permalink/')

        # Remove junk
        if '?' in link:
            link = link[:link.find('?')]
        if '&' in link:
            link = link[:link.find('&')]

        # Add trailing slash
        if not link.endswith('/'):
            link = link + '/'

        if '/permalink/' not in link and '/videos/' not in link:
            raise ValueError(link)
    else:
        link = ''

    return link

In [12]:
class TestFacebookLink(unittest.TestCase):
    '''Class to test formatFacebookLink function'''   

    def test_invalid_link(self):
        '''Test invalid link'''
        with self.assertRaises(ValueError):
            formatFacebookLink('http://youtube.com/')

    def test_permalink(self):
        '''Test permalink'''
        self.assertEqual(formatFacebookLink(
            'https://www.facebook.com/events/903549840109576/permalink/907939493003944/'),
            'https://www.facebook.com/events/903549840109576/permalink/907939493003944/')

    def test_video(self):
        '''Test video'''
        self.assertEqual(formatFacebookLink(
            'https://www.facebook.com/isak.majer/videos/3263767253848359/'),
            'https://www.facebook.com/isak.majer/videos/3263767253848359/')

    def test_post(self):
        '''Test post'''
        self.assertEqual(formatFacebookLink(
            'https://www.facebook.com/events/679860472562391/?post_id=681959239019181&view=permalink'),
            'https://www.facebook.com/events/679860472562391/permalink/681959239019181/')

    def test_mobile(self):
        '''Test mobile'''
        self.assertEqual(formatFacebookLink(
            'https://m.facebook.com/events/903549840109576?view=permalink&id=907264923071401/'),
            'https://www.facebook.com/events/903549840109576/permalink/907264923071401/')

    def test_junk(self):
        '''Test trailing junk'''
        self.assertEqual(formatFacebookLink(
            'https://www.facebook.com/isak.majer/videos/3263767253848359/?notif_id=1592251428027425&notif_t=feedback_reaction_generic'),
            'https://www.facebook.com/isak.majer/videos/3263767253848359/')

    def test_trailing_slash(self):
        '''Test trailing slash'''
        self.assertEqual(formatFacebookLink(
            'https://www.facebook.com/events/903549840109576/permalink/907939493003944'),
            'https://www.facebook.com/events/903549840109576/permalink/907939493003944/')

## calculateBest() and calculateAverage()

Functions to calculate best and average from list of solves

In [13]:
secsInDay = 86400

def calculateBest(solves):
    '''Calculate best result from list of solves'''

    best = -1
    for solve in solves:
        if solve > 0 and (best < 0 or solve < best):
            best = solve

    return best


def calculateAverage(solves):
    '''Calculate Mo3 / Ao5 from list of solves'''

    average = -1
    
    # Copy list of solves prior to manipulation
    tmpSolves = solves.copy()

    # Remove DNS - e.g. doing Mo3 rather than Ao5
    while -2 in tmpSolves:
        tmpSolves.remove(-2)

    # Convert DNF to 1 day
    for i in range(len(tmpSolves)):
        if tmpSolves[i] < 0:
            tmpSolves[i] = secsInDay

    # Sort solves
    tmpSolves = sorted(tmpSolves)

    # Calculate Ao5
    if (len(tmpSolves)) >= 4:
        if tmpSolves[3] == secsInDay:
            average = -1
        else:
            average = round((tmpSolves[1] + tmpSolves[2] + tmpSolves[3]) / 3, 2)

    # Calculate Mo3
    elif (len(tmpSolves)) == 3:
        if tmpSolves[2] == secsInDay:
            average = -1
        else:
            average = round((tmpSolves[0] + tmpSolves[1] + tmpSolves[2]) / 3, 2)

    return average

In [14]:
class TestCalculateBest(unittest.TestCase):
    '''Class to test calculateBest function'''   

    def test_simple_best(self):
        '''Simple test for all positions'''
        self.assertEqual(calculateBest([11, 12, 13, 14, 15]), 11)
        self.assertEqual(calculateBest([12, 11, 13, 14, 15]), 11)
        self.assertEqual(calculateBest([12, 13, 11, 14, 15]), 11)
        self.assertEqual(calculateBest([12, 13, 14, 11, 15]), 11)
        self.assertEqual(calculateBest([12, 13, 14, 15, 11]), 11)

    def test_ignore_dnf(self):
        '''Test to ignore DNF'''
        self.assertEqual(calculateBest([-1, 12, 13, 14, 15]), 12)
        self.assertEqual(calculateBest([12, -1, 13, 14, 15]), 12)

    def test_ignore_dns(self):
        '''Test to ignore DNS'''
        self.assertEqual(calculateBest([-2, 12, 13, 14, 15]), 12)
        self.assertEqual(calculateBest([12, -2, 13, 14, 15]), 12)

        
class TestCalculateAverage(unittest.TestCase):
    '''Class to test calculateAverage function'''   

    def test_simple_average(self):
        '''Simple test for various positions'''
        self.assertEqual(calculateAverage([11, 12, 13, 14, 15]), 13)
        self.assertEqual(calculateAverage([12, 13, 14, 15, 11]), 13)
        self.assertEqual(calculateAverage([13, 14, 15, 11, 12]), 13)
        self.assertEqual(calculateAverage([14, 15, 11, 12, 13]), 13)
        self.assertEqual(calculateAverage([15, 11, 12, 13, 14]), 13)

    def test_ignore_dnf(self):
        '''Test to ignore DNF'''
        self.assertEqual(calculateAverage([-1, 12, 13, 14, 15]), 14)
        self.assertEqual(calculateAverage([12, -1, 13, 14, 15]), 14)

    def test_ignore_dns(self):
        '''Test to ignore DNS'''
        self.assertEqual(calculateAverage([-2, 12, 13, 14, 15]), 14)
        self.assertEqual(calculateAverage([12, -2, 13, 14, 15]), 14)
        
    def test_dnf_average(self):
        '''Test to ignore DNF'''
        self.assertEqual(calculateAverage([-1, -1, 13, 14, 15]), -1)
        self.assertEqual(calculateAverage([12, -1, 13, 14, -1]), -1)

## getSafeName()

Function to convert name to regular ASCII for use in filenames and URLs

In [15]:
import unicodedata

def getSafeName(name):
    '''Return name which is safe for URL'''

    # Remove bracketed portion of name
    if '(' in name:
        name = name[:name.find('(')]

    nameDecomposed = unicodedata.normalize('NFKD', name)
    safeName = nameDecomposed.encode('ascii', 'ignore').decode('ascii').strip()
    
    if len(safeName) == 0:
        safeName = name

    return safeName.lower().replace(' ', '_').replace('-', '_').replace('.', '')

In [16]:
class TestGetSafeName(unittest.TestCase):
    '''Class to test getSafeName function'''   

    def test_spaces(self):
        '''Test spaces'''
        self.assertEqual(getSafeName('Michael George'), 'michael_george')

    def test_hyphens(self):
        '''Test hyphens'''
        self.assertEqual(getSafeName('Jan Adams-Fielding'), 'jan_adams_fielding')

    def test_dots(self):
        '''Test dots'''
        self.assertEqual(getSafeName('Joshua M. Woodward'), 'joshua_m_woodward')

    def test_accents(self):
        '''Test accents'''
        self.assertEqual(getSafeName('Markus Niederöst'), 'markus_niederost')
        self.assertEqual(getSafeName('Raúl Morales'), 'raul_morales')
        self.assertEqual(getSafeName('Shawn Boucké'), 'shawn_boucke')
        self.assertEqual(getSafeName('Zack Âû Black'), 'zack_au_black')

    def test_brackets(self):
        '''Test brackets'''
        self.assertEqual(getSafeName('Jamie Brady (Deansie)'), 'jamie_brady')
        self.assertEqual(getSafeName('Jang Junho (장준호)'), 'jang_junho')

## writeGoogleSiteTag()

Site visitors are tracked using Google analytics

In [17]:
def writeGoogleSiteTag(f):
    '''Write Google Site Tag'''

    comment = '<!-- Global site tag (gtag.js) - Google Analytics -->'
    src = '<script async src="https://www.googletagmanager.com/gtag/js?id=UA-86348435-3"></script>'
    gtag = "<script>window.dataLayer = window.dataLayer || []; function gtag() {dataLayer.push(arguments);} gtag('js', new Date()); gtag('config', 'UA-86348435-3');</script>"

    f.write('\n{}\n{}\n{}\n'.format(comment, src, gtag))

## Run Unit Tests

In [18]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........................................
----------------------------------------------------------------------
Ran 42 tests in 0.076s

OK
